# 8-3 경량 모델(sLM) 전환 및 활용 방법

## 8-16 gpt-4o-mini활용 공정시장가액비율 파악     

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI

small_llm = ChatOpenAI(model="gpt-4o-mini")
search = DuckDuckGoSearchRun()

question = "10억짜리 집을 2채 가지고 있을 때 세금을 얼마나 내나요?"
market_value_rate_search = search.invoke(f"{datetime.now().year}년도 공정시장가액비율은?")
market_value_rate_prompt = PromptTemplate.from_template("""아래 [Context]는 공정시장가액비율에 관한 내용입니다. 
당신에게 주어진 공정시장가액비율에 관한 내용을 기반으로, 사용자의 상황에 대한 공정시장가액비율을 알려주세요.
별도의 설명 없이 공정시장가액비율만 반환해주세요.

[Context]
{context}

[Question]
질문: {question}
답변: 
""")

market_value_rate_chain = (
    market_value_rate_prompt
    | small_llm  # 기존 코드에서 llm만 small_llm으로 변경. LangChain의 추상화를 통해 사용하는 모델을 변경하면서 쉽게 디버깅이 가능하다.
    | StrOutputParser()
)
market_value_rate = market_value_rate_chain.invoke({'context': market_value_rate_search, 'question': question})
market_value_rate


'공정시장가액비율 60%'

## 8-17 토큰 사용량 확인을 위한 StrOutputParser() 제거

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

market_value_rate_search = search.invoke(f"{datetime.now().year}년도 공정시장가액비율은?")
market_value_rate_search
market_value_rate_prompt = PromptTemplate.from_template("""아래 [Context]는 공정시장가액비율에 관한 내용입니다. 
당신에게 주어진 공정시장가액비율에 관한 내용을 기반으로, 사용자의 상황에 대한 공정시장가액비율을 알려주세요.
별도의 설명 없이 공정시장가액비율만 반환해주세요.

[Context]
{context}

[Question]
질문: {question}
답변: 
""")

market_value_rate_chain = (
    market_value_rate_prompt
    | small_llm  # 기존 코드에서 llm만 small_llm으로 변경
                 # StrOutputParser()를 제거해서 토큰 사용량을 확인한다
)
market_value_rate = market_value_rate_chain.invoke({'context': market_value_rate_search, 'question': question})
market_value_rate


AIMessage(content='공정시장가액비율 60%', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 607, 'total_tokens': 618, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BSIeNJAZkd1Lo0xviQ7SnEc8j2II8', 'finish_reason': 'stop', 'logprobs': None}, id='run-56e211b8-61be-4400-948c-d90c23cb6463-0', usage_metadata={'input_tokens': 607, 'output_tokens': 11, 'total_tokens': 618, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI

search = DuckDuckGoSearchRun()
llm = ChatOpenAI(model="gpt-4o")

market_value_rate_search = search.invoke(f"{datetime.now().year}년도 공정시장가액비율은?")
market_value_rate_search
market_value_rate_prompt = PromptTemplate.from_template("""아래 [Context]는 공정시장가액비율에 관한 내용입니다. 
당신에게 주어진 공정시장가액비율에 관한 내용을 기반으로, 사용자의 상황에 대한 공정시장가액비율을 알려주세요.
별도의 설명 없이 공정시장가액비율만 반환해주세요.

[Context]
{context}

[Question]
질문: {question}
답변: 
""")

market_value_rate_chain = (
    market_value_rate_prompt
    | llm  # small_llm 대신 llm을 사용
)
market_value_rate = market_value_rate_chain.invoke({'context': market_value_rate_search, 'question': question})
market_value_rate


AIMessage(content='60%', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 607, 'total_tokens': 610, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BSIfiBeYGtb4MgEjQIcSrqHIrNSbM', 'finish_reason': 'stop', 'logprobs': None}, id='run-4878f144-1bf7-4e5e-8762-ddf9b5a8e5e2-0', usage_metadata={'input_tokens': 607, 'output_tokens': 3, 'total_tokens': 610, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
# 필요한 라이브러리들을 가져온다
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma

# 환경변수 파일(.env)을 불러온다
load_dotenv()

# Upstage의 SOLAR 임베딩 모델을 초기화한다
embedding = UpstageEmbeddings(
    model='solar-embedding-1-large'  
)

# Chroma 벡터 저장소를 초기화한다
vector_store = Chroma(
                  embedding_function=embedding,
                  collection_name='tax-markdown', 
                  persist_directory="./tax-markdown")

# retriever를 설정하고 상위 3개 결과를 반환하도록 한다
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [7]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
rag_prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

tax_base_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm  # 토큰 사용량 확인을 위해 마지막에 StrOutputParser()를 제거한다.
)
tax_base_question = '주택에 대한 종합부동산세 과세표준을 계산하는 방법은 무엇인가요? 수식으로 표현해서 수식만 반환해주세요'
tax_base_response = tax_base_chain.invoke(tax_base_question)
tax_base_response


/Users/jasonkang/personal/langgraph-book/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


AIMessage(content='과세표준 = (공시가격의 합계 - 공제금액) × 공정시장가액비율', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 2613, 'total_tokens': 2639, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BSIgx38fmvWr4hTevCdh3B1LXwJhG', 'finish_reason': 'stop', 'logprobs': None}, id='run-ffc68e73-668e-4012-827c-6713d2b6c8c7-0', usage_metadata={'input_tokens': 2613, 'output_tokens': 26, 'total_tokens': 2639, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})